In [1]:
import numpy as np
import gym
import copy
import renom as rm
from renom_rl.discrete.a2c import A2C
from renom_rl.environ.env import BaseEnv
from tqdm import tqdm
from renom_rl.utility import Animation

In [2]:
env=gym.make("CartPole-v0")

    
class ActorCritic(rm.Model):
    def __init__(self):
        self.l1=rm.Dense(32)
        self.l2=rm.Dense(32)
        self.l3=rm.Dense(2)
        self.l4=rm.Dense(1)
        
    def forward(self,x):
        h1 = self.l1(x)
        h2 = self.l2(h1)
        act = rm.softmax(self.l3(h2))
        val=self.l4(h2)
        return act,val
    
model=ActorCritic()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/mikeogawa/gym/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


In [3]:
class CustomEnv(BaseEnv):
    def __init__(self,env):
        self.env = env
        self.action_shape = (2,)
        self.state_shape = (4,)
        self.continous_step = 0
        self.animation=Animation()
        self.test=False

    def sample(self):
        return self.env.action_space.sample()

    def step(self, action):
        state, _ , terminal,_ = self.env.step(action)
        
        reward=0
        self.continous_step += 1
        
        if terminal:
            if self.continous_step > 195:
                reward=1
            else:
                reward=-1
            
            self.continous_step = 0
        
        return state, reward, terminal
    
    def test_start(self):
        if self.test:
            self.animation.reset()
            self.env.reset()

    def test_epoch_step(self):
        if self.test:
            self.animation.store(self.env.render(mode="rgb_array"))

    def test_close(self):
        if self.test:
            self.env.close()
            self.env.viewer=None
 
    def reset_anime(self):
        self.animation.reset()
    
    def reset(self):
        return self.env.reset()

In [ ]:
Env=CustomEnv(env)
a2c=A2C(Env,model)

In [ ]:
a2c.fit(
    epoch=1,
    epoch_step=10000)

agent0, epoch 0001 loss 0.0159 rewards in epoch -40.000  episode 40.0 rewards in episode -1.000.:   7%|▋         | 670/10000 [00:02<00:42, 218.60it/s]

In [ ]:
Env.test=True
a2c.test()

In [ ]:
Env.animation.run()
Env.animation.reset()